In [69]:
# ref  : https://codelabs.developers.google.com/vertex-mlmd-pipelines#3

In [1]:
!gcloud config list

[core]
account = argolis-demo@senchan.altostrat.com
disable_usage_reporting = True
project = argolis-demo-project

Your active configuration is: [default]


In [2]:
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [3]:
import matplotlib.pyplot as plt
import pandas as pd

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath

from google.cloud import aiplatform

# We'll use this namespace for metadata querying
from google.cloud import aiplatform_v1

In [37]:
@component(
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow", "db-dtypes"],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml"
)
def get_dataframe(
    bq_table: str,
    output_data_path: OutputPath("Dataset")
):
    from google.cloud import bigquery
    import pandas as pd

    bqclient = bigquery.Client(project="argolis-demo-project")
    table = bigquery.TableReference.from_string(
        bq_table
    )
    rows = bqclient.list_rows(
        table
    )
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe = dataframe.sample(frac=1, random_state=2)
    dataframe.to_csv(output_data_path, index=False)

In [17]:
bq_table = "argolis-demo-project.bigquery_handson.beans"

In [110]:
project_number = 278305018396

In [34]:
from google.cloud import bigquery
import pandas as pd

bqclient = bigquery.Client(project="argolis-demo-project")
table = bigquery.TableReference.from_string(
    bq_table
)
rows = bqclient.list_rows(
    table
)
dataframe = rows.to_dataframe(
    create_bqstorage_client=True,
)
dataframe = dataframe.sample(frac=1, random_state=2)
dataframe.to_csv('bean_sample.csv', index=False)

In [35]:
import pandas as pd
df = pd.read_csv('bean_sample.csv')

In [36]:
df.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,46892,834.759,330.650375,181.543648,1.821327,0.835789,47506,244.345552,0.712255,0.987075,0.845642,0.738985,0.007051,0.001297,0.546098,0.994624,HOROZ
1,37720,706.226,238.253632,201.964767,1.179679,0.530495,38103,219.149710,0.755427,0.989948,0.950373,0.919817,0.006316,0.002789,0.846063,0.998083,SEKER
2,44838,807.043,300.736509,190.773486,1.576406,0.773042,45454,238.934122,0.824379,0.986448,0.865093,0.794497,0.006707,0.001648,0.631225,0.995067,SIRA
3,34671,674.874,235.589519,187.649724,1.255475,0.604624,35015,210.105898,0.765770,0.990176,0.956601,0.891830,0.006795,0.002652,0.795361,0.998556,SEKER
4,71601,1107.086,399.311734,229.197436,1.742217,0.818868,73088,301.935796,0.643801,0.979655,0.734119,0.756141,0.005577,0.001125,0.571749,0.996110,BARBUNYA


In [29]:
!pwd

/home/jupyter/vertex-ai-samples/notebooks/official


In [19]:
dataframe.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [22]:
labels = dataframe.pop("Class").tolist()

In [25]:
dataframe.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166


In [5]:
@component(
    packages_to_install=["sklearn", "pandas", "joblib", "db-dtypes"],
    base_image="python:3.9",
    output_component_file="beans_model_component.yaml",
)
def sklearn_train(
    dataset: Input[Dataset],
    metrics: Output[Metrics],
    model: Output[Model]
):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump

    import pandas as pd
    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)

    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train,y_train)
    score = skmodel.score(x_test,y_test)
    print('accuracy is:',score)

    metrics.log_metric("accuracy",(score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))
    dump(skmodel, model.path + ".joblib")

In [6]:
@component(
    packages_to_install=["google-cloud-aiplatform"],
    base_image="python:3.9",
    output_component_file="beans_deploy_component.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=region)

    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri = model.uri.replace("model", ""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")

    # Save data to the output params
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

In [7]:
BUCKET_NAME = 'argolis-demo-senchan'

In [8]:
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

'gs://argolis-demo-senchan/pipeline_root/'

In [9]:
PROJECT_ID = "argolis-demo-project"  # @param {type:"string"}

In [10]:
REGION = "us-central1"  # @param {type: "string"}

In [11]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [12]:
@pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="mlmd-pipeline",
)
def pipeline(
    bq_table: str = "",
    output_data_path: str = "data.csv",
    project: str = PROJECT_ID,
    region: str = REGION
):
    dataset_task = get_dataframe(bq_table)

    model_task = sklearn_train(
        dataset_task.output
    )

    deploy_task = deploy_model(
        model=model_task.outputs["model"],
        project=project,
        region=region
    )

In [13]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="mlmd_pipeline.json"
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [14]:
run1 = aiplatform.PipelineJob(
    display_name="mlmd-pipeline",
    template_path="mlmd_pipeline.json",
    job_id="mlmd-pipeline-small-{0}".format(TIMESTAMP),
    parameter_values={"bq_table": "argolis-demo-project.bigquery_handson.beans"},
    enable_caching=True,
)

In [15]:
run1.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/278305018396/locations/us-central1/pipelineJobs/mlmd-pipeline-small-20220530235509
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/278305018396/locations/us-central1/pipelineJobs/mlmd-pipeline-small-20220530235509')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mlmd-pipeline-small-20220530235509?project=278305018396


# Titanic version

In [191]:
from datetime import datetime
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)

from kfp.v2 import compiler
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip

In [192]:
import matplotlib.pyplot as plt
import pandas as pd

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath

from google.cloud import aiplatform

# We'll use this namespace for metadata querying
from google.cloud import aiplatform_v1

In [193]:
@component(
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow", "db-dtypes"],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml"
)
def get_dataframe(
    bq_table: str,
    output_data_path: OutputPath("Dataset")
):
    from google.cloud import bigquery
    import pandas as pd

    bqclient = bigquery.Client(project="argolis-demo-project")
    table = bigquery.TableReference.from_string(
        bq_table
    )
    rows = bqclient.list_rows(
        table
    )
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe = dataframe.sample(frac=1, random_state=2)
    dataframe.to_csv(output_data_path, index=False)

In [194]:
@component(
    packages_to_install=["sklearn", "pandas", "joblib", "db-dtypes"],
    base_image="python:3.9",
    output_component_file="titanic_model_component.yaml",
)
def sklearn_train(
    dataset: Input[Dataset],
    metrics: Output[Metrics],
    model: Output[Model]
):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump

    import pandas as pd
    df = pd.read_csv(dataset.path)
    df = df.drop(["name","ticket","cabin","boat","body","home_dest"], axis=1)
    df["sex"] = df["sex"].map({"male":0,"female":1})
    df = df[df.fare != '?']
    df = df[df.embarked != '?']
    df = df[df.age != '?']
    df = pd.get_dummies(df, columns=['embarked'])
    labels = df.pop("survived").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)

    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train,y_train)
    score = skmodel.score(x_test,y_test)
    print('accuracy is:',score)

    metrics.log_metric("accuracy",(score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))
    dump(skmodel, model.path + ".joblib")

In [195]:
@component(
    packages_to_install=["google-cloud-aiplatform"],
    base_image="python:3.9",
    output_component_file="titanic_deploy_component.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=region)

    deployed_model = aiplatform.Model.upload(
        display_name="titanic-model-pipeline",
        artifact_uri = model.uri.replace("model", ""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")

    # Save data to the output params
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

In [196]:
BUCKET_NAME = 'argolis-demo-senchan'
PIPELINE_ROOT = f"gs://{BUCKET_NAME}/pipeline_root/"
PROJECT_ID = "argolis-demo-project"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [197]:
@pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="titanic-pipeline",
)
def pipeline(
    bq_table: str = "",
    output_data_path: str = "data.csv",
    project: str = PROJECT_ID,
    region: str = REGION
):
    dataset_task = get_dataframe(bq_table)

    model_task = sklearn_train(
        dataset_task.output
    )

    deploy_task = deploy_model(
        model=model_task.outputs["model"],
        project=project,
        region=region
    )

In [198]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="titanic_pipeline.json"
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [199]:
titanic_run = aiplatform.PipelineJob(
    display_name="titanic-pipeline",
    template_path="titanic_pipeline.json",
    job_id="titanic-pipeline-{0}".format(TIMESTAMP),
    parameter_values={"bq_table": "argolis-demo-project.bigquery_handson.survivors"},
    enable_caching=False,
)

In [200]:
titanic_run.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/278305018396/locations/us-central1/pipelineJobs/titanic-pipeline-20220531030023
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/278305018396/locations/us-central1/pipelineJobs/titanic-pipeline-20220531030023')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/titanic-pipeline-20220531030023?project=278305018396


# Workspace

In [74]:
bq_table = "argolis-demo-project.bigquery_handson.survivors"

In [53]:
output_data_path = 'titanic.csv'

In [75]:
bqclient = bigquery.Client(project="argolis-demo-project")
table = bigquery.TableReference.from_string(
    bq_table
)
rows = bqclient.list_rows(
    table
)
dataframe = rows.to_dataframe(
    create_bqstorage_client=True,
)
dataframe = dataframe.sample(frac=1, random_state=2)
dataframe.to_csv(output_data_path, index=False)

In [133]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from joblib import dump

import pandas as pd

In [130]:
df = pd.read_csv(output_data_path)
df = df.drop(["name","ticket","cabin","boat","body","home_dest"], axis=1)
df["sex"] = df["sex"].map({"male":0,"female":1})
df = df[df.fare != '?']
df = df[df.embarked != '?']
df = df[df.age != '?']
df = pd.get_dummies(df, columns=['embarked'])
labels = df.pop("survived").tolist()
data = df.values.tolist()
x_train, x_test, y_train, y_test = train_test_split(data, labels)

In [142]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,3,0,18,0,0,8.6625,0,0,1
1,2,0,35,0,0,26,0,0,1
2,2,1,35,0,0,21,0,0,1
3,3,0,45.5,0,0,7.225,1,0,0
4,2,1,36,0,0,13,0,0,1


In [131]:
skmodel = DecisionTreeClassifier()
skmodel.fit(x_train,y_train)
score = skmodel.score(x_test,y_test)

In [135]:
print('accuracy is:',score)

metrics.log_metric("accuracy",(score * 100.0))
metrics.log_metric("framework", "Scikit Learn")
metrics.log_metric("dataset_size", len(df))
dump(skmodel, model.path + ".joblib")

accuracy is: 0.7662835249042146


NameError: name 'metrics' is not defined